In [1]:
pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.ticker as mtick

import itertools
import s3fs
import os
import torch
import random
import json
# import boto
import scipy
# import sktime
import trino

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from trino.dbapi import connect
from trino.auth import OAuth2Authentication, BasicAuthentication

from datetime import datetime
# from config import Config  
from collections import defaultdict, Counter
from tqdm import tqdm 
from itertools import zip_longest
from contextlib import closing

tqdm.pandas()

In [7]:
conn = connect(
           host=f'trino.prd.rgn',
           port=443,
           user="roan.schellingerhout@randstadgroep.nl",
           catalog='iceberg',
           verify="certificate",
           http_scheme="https",
           auth=OAuth2Authentication()
     )

In [8]:
cur = conn.cursor()

In [ ]:
vacancy_data = cur.execute("""SELECT DISTINCT * 
                              FROM datalake_prd_current_iceberg.vacancypublishingsystem_vacancies""").fetchall()

In [12]:
headers = [i[0] for i in cur.description]

headers

['operationtype',
 'createddate',
 'modifieddate',
 'vacancynumber',
 'versionnumber',
 'publicationstatus',
 'publicationstartdate',
 'publicationenddate',
 'vacancytype',
 'requestsourcetype',
 'requestsourceidentifier',
 'requestsourceclientnumber',
 'unitabbreviation',
 'unitnumber',
 'unitname',
 'opco',
 'maxapplicants',
 'contracttype',
 'contracttypecode',
 'salarytype',
 'salarytypecode',
 'educationlevel',
 'educationlevelcode',
 'experience',
 'experiencecode',
 'fieldofwork',
 'fieldofworkcode',
 'functiongroup',
 'functiongroupcode',
 'industrycode',
 'industrylabel',
 'functioncode',
 'functionlabel',
 'titlefromrequest',
 'vacancytitle',
 'clientname',
 'vacancyclientname',
 'worklocationcity',
 'worklocationpostalcode',
 'worklocationlatitude',
 'worklocationlongitude',
 'clientshowlogo',
 'clientshowname',
 'maxtraveldistance',
 'minsalary',
 'maxsalary',
 'hidesalary',
 'minhoursperweek',
 'maxhoursperweek',
 'mindaysperweek',
 'maxdaysperweek',
 'jobstartdate',
 'job

In [7]:
df = pd.DataFrame(data=vacancy_data, columns=headers)[['createddate', 'vacancynumber', 'versionnumber', 'vacancytype', 'requestsourcetype',
                                                       'requestsourceidentifier', 'unitname', 'opco', 'maxapplicants', 'contracttype', 'salarytype',
                                                       'educationlevel', 'experience', 'fieldofwork', 'functiongroup', 'industrylabel', 'functionlabel',
                                                       'titlefromrequest', 'vacancytitle', 'clientname', 'vacancyclientname', 'worklocationcity',
                                                       'worklocationpostalcode', 'worklocationlatitude', 'worklocationlongitude', 'clientshowlogo',
                                                       'clientshowname', 'maxtraveldistance', 'minsalary', 'maxsalary', 'hidesalary', 'minhoursperweek',
                                                       'maxhoursperweek', 'mindaysperweek', 'maxdaysperweek', 'jobstartdate', 'jobenddate', 'shiftwork',
                                                       'nightshift', 'weekendshift', 'vacationwork', 'jobboardtags', 'textvacancyfromoldvms', 'introductorytext',
                                                       'benefits', 'skillsintro', 'skillsbullets', 'taskintro', 'taskbullets', 'employerintro', 'employerbullets',
                                                       'callforaction', 'contactdisplayname', 'contactfunction', 'contactfreetext', 'contactphonenumber',
                                                       'contactemail', 'pagetextanalysisscore', 'pagetextanalysisreadinglevel', 'vacancylanguagecode', 'sbilabel']]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173221 entries, 0 to 173220
Data columns (total 61 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   createddate                   173221 non-null  datetime64[ns]
 1   vacancynumber                 173221 non-null  int64         
 2   versionnumber                 173221 non-null  int64         
 3   vacancytype                   173221 non-null  object        
 4   requestsourcetype             173221 non-null  object        
 5   requestsourceidentifier       173221 non-null  object        
 6   unitname                      173221 non-null  object        
 7   opco                          173221 non-null  object        
 8   maxapplicants                 173221 non-null  int64         
 9   contracttype                  173221 non-null  object        
 10  salarytype                    173221 non-null  object        
 11  educationleve

In [9]:
# Template string
template = """Title: {vacancytitle}
Company: {clientname}

Introduction:\n {introductorytext}

Benefits:\n {benefits}

Skills:\n {skillsintro}\n {skillsbullets}

Task:\n {taskintro}\n {taskbullets}

Employer: {employerintro}\n {employerbullets}

{callforaction}

Contact:\n {contactfreetext}
"""

df["listing"] = df.progress_apply(lambda x: template.format(**x[["vacancytitle", "clientname", "introductorytext", "benefits", "skillsintro", 
                                                                 "skillsbullets", "taskintro", "taskbullets", "employerintro", "employerbullets", 
                                                                 "callforaction", "contactfreetext"]]), axis=1)

100%|██████████| 173221/173221 [01:53<00:00, 1519.82it/s]


In [14]:
df[list(set(df.columns) - set(["vacancytitle", "clientname", "introductorytext", "benefits", "skillsintro", "skillsbullets", "taskintro", "taskbullets", "employerintro", "employerbullets", "callforaction", "contactfreetext"]))].to_csv("./cleaner_data/vacancies.csv")